In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Ver dimensões
print("Train:", train.shape)
print("Test:", test.shape)

# Olhar primeiras linhas
print(train.head())

# Ver estatísticas gerais
print(train.describe(include="all"))


Train: (646, 33)
Test: (277, 32)
    id  age_first_funding_year  age_last_funding_year  \
0  719                   10.42                  13.09   
1  429                    3.79                   3.79   
2  178                    0.71                   2.28   
3  197                    3.00                   5.00   
4  444                    0.66                   5.88   

   age_first_milestone_year  age_last_milestone_year  relationships  \
0                      8.98                    12.72              4   
1                       NaN                      NaN             21   
2                      1.95                     2.28              5   
3                      9.62                    10.39             16   
4                      6.21                     8.61             29   

   funding_rounds  funding_total_usd  milestones  is_CA  ...  is_consulting  \
0               3            4087500           3      1  ...              0   
1               1           45000000   

In [77]:
# ANÁLISE EXPLORATÓRIA INICIAL
# Verificar informações gerais dos dados
print("=== INFORMAÇÕES GERAIS ===")
print("Train info:")
print(train.info())
print("\nTest info:")
print(test.info())

# Verificar valores ausentes
print("\n=== VALORES AUSENTES ===")
print("Train - valores nulos:")
print(train.isnull().sum())
print("\nTest - valores nulos:")
print(test.isnull().sum())

# Verificar duplicatas
print("\n=== DUPLICATAS ===")
print(f"Train duplicatas: {train.duplicated().sum()}")
print(f"Test duplicatas: {test.duplicated().sum()}")

# Analisar a variável target (se existir)
if 'target' in train.columns or 'success' in train.columns:
    target_col = 'target' if 'target' in train.columns else 'success'
    print(f"\n=== DISTRIBUIÇÃO DO TARGET ({target_col}) ===")
    print(train[target_col].value_counts())
    print(f"Proporção: {train[target_col].value_counts(normalize=True)}")


=== INFORMAÇÕES GERAIS ===
Train info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate             

In [78]:
#LIMPEZA E PROCESSAMENTO DOS DADOS

# 1. Remover duplicatas (se houver)
train = train.drop_duplicates()
test = test.drop_duplicates()

# 2. Identificar tipos de colunas
print("\n=== TIPOS DE COLUNAS ===")
numerical_cols = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

print(f"Colunas numéricas: {numerical_cols}")
print(f"Colunas categóricas: {categorical_cols}")




=== TIPOS DE COLUNAS ===
Colunas numéricas: ['id', 'age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants', 'labels']
Colunas categóricas: ['category_code']


In [79]:
# 3. Tratamento de valores ausentes
def tratar_valores_ausentes(df, is_train=True):
    """Função para tratar valores ausentes"""
    df_clean = df.copy()
    
    # Para colunas numéricas: preencher com mediana
    for col in numerical_cols:
        if col in df_clean.columns and df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
    
    # Para colunas categóricas: preencher com moda ou 'Unknown'
    for col in categorical_cols:
        if col in df_clean.columns and df_clean[col].isnull().sum() > 0:
            # Se for poucas categorias, usar moda; senão usar 'Unknown'
            if df_clean[col].nunique() < 20:
                df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
            else:
                df_clean[col].fillna('Unknown', inplace=True)
    
    return df_clean

# Aplicar tratamento
train_clean = tratar_valores_ausentes(train, is_train=True)
test_clean = tratar_valores_ausentes(test, is_train=False)


In [80]:
# 4. Análise de outliers (para colunas numéricas)
def detectar_outliers(df, cols):
    """Detecta outliers usando IQR"""
    outliers_info = {}
    
    for col in cols:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            outliers_info[col] = {
                'count': len(outliers),
                'percentage': len(outliers) / len(df) * 100,
                'bounds': (lower_bound, upper_bound)
            }
    
    return outliers_info

# Detectar outliers
outliers_train = detectar_outliers(train_clean, numerical_cols)
print("\n=== OUTLIERS DETECTADOS ===")
for col, info in outliers_train.items():
    print(f"{col}: {info['count']} outliers ({info['percentage']:.2f}%)")


=== OUTLIERS DETECTADOS ===
id: 0 outliers (0.00%)
age_first_funding_year: 20 outliers (3.10%)
age_last_funding_year: 11 outliers (1.70%)
age_first_milestone_year: 41 outliers (6.35%)
age_last_milestone_year: 22 outliers (3.41%)
relationships: 47 outliers (7.28%)
funding_rounds: 10 outliers (1.55%)
funding_total_usd: 50 outliers (7.74%)
milestones: 0 outliers (0.00%)
is_CA: 0 outliers (0.00%)
is_NY: 71 outliers (10.99%)
is_MA: 61 outliers (9.44%)
is_TX: 24 outliers (3.72%)
is_otherstate: 136 outliers (21.05%)
is_software: 105 outliers (16.25%)
is_web: 97 outliers (15.02%)
is_mobile: 65 outliers (10.06%)
is_enterprise: 53 outliers (8.20%)
is_advertising: 45 outliers (6.97%)
is_gamesvideo: 37 outliers (5.73%)
is_ecommerce: 20 outliers (3.10%)
is_biotech: 25 outliers (3.87%)
is_consulting: 2 outliers (0.31%)
is_othercategory: 0 outliers (0.00%)
has_VC: 0 outliers (0.00%)
has_angel: 0 outliers (0.00%)
has_roundA: 0 outliers (0.00%)
has_roundB: 0 outliers (0.00%)
has_roundC: 152 outliers (

In [81]:
# 5. Codificação de variáveis categóricas - VERSÃO CORRIGIDA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def processar_categoricas(train_df, test_df, categorical_columns):
    """Processa variáveis categóricas - versão que lida com categorias novas no test"""
    train_processed = train_df.copy()
    test_processed = test_df.copy()
    
    for col in categorical_columns:
        if col in train_processed.columns:
            print(f"\nProcessando coluna: {col}")
            
            # Se poucas categorias, usar One-Hot Encoding
            if train_processed[col].nunique() <= 10:
                print(f"  → One-Hot Encoding ({train_processed[col].nunique()} categorias)")
                
                # Identificar todas as categorias únicas entre train e test
                all_categories = set(train_processed[col].unique()) | set(test_processed[col].unique())
                
                # One-Hot Encoding
                train_dummies = pd.get_dummies(train_processed[col], prefix=col)
                test_dummies = pd.get_dummies(test_processed[col], prefix=col)
                
                # Garantir mesmas colunas em train e test
                for category in all_categories:
                    dummy_col = f"{col}_{category}"
                    if dummy_col not in train_dummies.columns:
                        train_dummies[dummy_col] = 0
                    if dummy_col not in test_dummies.columns:
                        test_dummies[dummy_col] = 0
                
                # Ordenar colunas para garantir mesma ordem
                dummy_columns = sorted(train_dummies.columns)
                train_dummies = train_dummies[dummy_columns]
                test_dummies = test_dummies[dummy_columns]
                
                # Remover coluna original e adicionar dummies
                train_processed = train_processed.drop(col, axis=1)
                test_processed = test_processed.drop(col, axis=1)
                train_processed = pd.concat([train_processed, train_dummies], axis=1)
                test_processed = pd.concat([test_processed, test_dummies], axis=1)
            
            else:
                print(f"  → Label Encoding ({train_processed[col].nunique()} categorias)")
                
                # CORREÇÃO ROBUSTA: Lidar com categorias novas no test
                # 1. Converter para string primeiro
                train_processed[col] = train_processed[col].astype(str)
                test_processed[col] = test_processed[col].astype(str)
                
                # 2. Obter todas as categorias únicas
                train_categories = set(train_processed[col].unique())
                test_categories = set(test_processed[col].unique())
                
                # 3. Categorias que estão no test mas não no train
                new_categories = test_categories - train_categories
                if new_categories:
                    print(f"    ⚠️  Categorias novas no test: {new_categories}")
                    # Mapear categorias novas para 'Unknown'
                    for new_cat in new_categories:
                        test_processed.loc[test_processed[col] == new_cat, col] = 'Unknown'
                    
                    # ADICIONANDO 'Unknown' ao train se necessário
                    if 'Unknown' not in train_categories:
                        # Adicionar 'Unknown' a algumas linhas do train
                        n_unknown = min(5, len(train_processed) // 100)  # 1% ou 5 linhas, o que for menor
                        unknown_indices = train_processed.sample(n=n_unknown).index
                        train_processed.loc[unknown_indices, col] = 'Unknown'
                        print(f"    📝 Adicionadas {n_unknown} linhas 'Unknown' ao train")
                
                # 4. Aplicar Label Encoding com tratamento de erro
                try:
                    le = LabelEncoder()
                    train_processed[col] = le.fit_transform(train_processed[col])
                    test_processed[col] = le.transform(test_processed[col])
                except ValueError as e:
                    print(f"    🚨 Erro no Label Encoding para {col}: {e}")
                    # Fallback: usar One-Hot mesmo com muitas categorias
                    print(f"    🔄 Usando One-Hot como fallback")
                    train_dummies = pd.get_dummies(train_processed[col], prefix=col)
                    test_dummies = pd.get_dummies(test_processed[col], prefix=col)
                    
                    # Garantir mesmas colunas
                    all_columns = set(train_dummies.columns) | set(test_dummies.columns)
                    for dummy_col in all_columns:
                        if dummy_col not in train_dummies.columns:
                            train_dummies[dummy_col] = 0
                        if dummy_col not in test_dummies.columns:
                            test_dummies[dummy_col] = 0
                    
                    # Substituir coluna
                    train_processed = train_processed.drop(col, axis=1)
                    test_processed = test_processed.drop(col, axis=1)
                    train_processed = pd.concat([train_processed, train_dummies], axis=1)
                    test_processed = pd.concat([test_processed, test_dummies], axis=1)
    
    return train_processed, test_processed

# Aplicar processamento categórico com correção
train_processed, test_processed = processar_categoricas(train_clean, test_clean, categorical_cols)


Processando coluna: category_code
  → Label Encoding (34 categorias)
    ⚠️  Categorias novas no test: {'hospitality'}
    📝 Adicionadas 5 linhas 'Unknown' ao train


In [82]:
# 6. Normalização/Padronização de variáveis numéricas
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def normalizar_dados(train_df, test_df, numerical_columns, method='standard'):
    """Normaliza dados numéricos"""
    train_norm = train_df.copy()
    test_norm = test_df.copy()
    
    # Verificar quais colunas numéricas ainda existem após processamento
    existing_num_cols = [col for col in numerical_columns if col in train_norm.columns and col in test_norm.columns]
    
    print(f"Colunas numéricas para normalizar: {existing_num_cols}")
    
    if method == 'standard':
        scaler = StandardScaler()
    else:
        scaler = MinMaxScaler()
    
    for col in existing_num_cols:
        if col in train_norm.columns and col in test_norm.columns:
            # Normalizar cada coluna individualmente
            scaler_col = StandardScaler() if method == 'standard' else MinMaxScaler()
            train_norm[col] = scaler_col.fit_transform(train_norm[[col]]).ravel()
            test_norm[col] = scaler_col.transform(test_norm[[col]]).ravel()
            print(f"  ✅ {col} normalizado")
    
    return train_norm, test_norm

# Aplicar normalização usando lista atualizada
train_final, test_final = normalizar_dados(train_processed, test_processed, numerical_cols)

Colunas numéricas para normalizar: ['id', 'age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants']
  ✅ id normalizado
  ✅ age_first_funding_year normalizado
  ✅ age_last_funding_year normalizado
  ✅ age_first_milestone_year normalizado
  ✅ age_last_milestone_year normalizado
  ✅ relationships normalizado
  ✅ funding_rounds normalizado
  ✅ funding_total_usd normalizado
  ✅ milestones normalizado
  ✅ is_CA normalizado
  ✅ is_NY normalizado
  ✅ is_MA normalizado
  ✅ is_TX normalizado
  ✅ is_otherstate normalizado
  ✅ is_software normalizado
  ✅ is_web normalizado
  ✅ i

In [83]:
# Verificar estado dos dados após processamento categórico
print("=== VERIFICAÇÃO APÓS PROCESSAMENTO CATEGÓRICO ===")
print(f"Train shape: {train_processed.shape}")
print(f"Test shape: {test_processed.shape}")
print(f"Colunas train: {list(train_processed.columns)}")
print(f"Colunas test: {list(test_processed.columns)}")

=== VERIFICAÇÃO APÓS PROCESSAMENTO CATEGÓRICO ===
Train shape: (646, 33)
Test shape: (277, 32)
Colunas train: ['id', 'age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants', 'labels']
Colunas test: ['id', 'age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce'

In [84]:
# VERIFICAÇÃO FINAL COMPLETA
print("=== VERIFICAÇÃO FINAL DOS DADOS PROCESSADOS ===")

# 1. Verificar shapes
print(f"📏 Train shape: {train_final.shape}")
print(f"📏 Test shape: {test_final.shape}")

# 2. Verificar se as colunas são iguais (exceto target)
train_cols = set(train_final.columns)
test_cols = set(test_final.columns)

# Identificar possível coluna target
possible_targets = ['labels', 'target', 'success', 'y']
target_col = None
for col in possible_targets:
    if col in train_cols and col not in test_cols:
        target_col = col
        break

if target_col:
    print(f"🎯 Target identificado: {target_col}")
    train_cols.remove(target_col)
else:
    print("⚠️  Target não identificado claramente")

# 3. Verificar compatibilidade das colunas
print(f"🔄 Colunas em comum: {len(train_cols & test_cols)}")
print(f"❌ Só no train: {train_cols - test_cols}")
print(f"❌ Só no test: {test_cols - train_cols}")

# 4. Verificar valores ausentes
print(f"\n🔍 Valores nulos train: {train_final.isnull().sum().sum()}")
print(f"🔍 Valores nulos test: {test_final.isnull().sum().sum()}")

# 5. Verificar tipos de dados
print(f"\n📊 Tipos de dados train:")
print(train_final.dtypes.value_counts())

# 6. Preparar dados para modelagem
if target_col:
    # Separar features e target
    X_train = train_final.drop(target_col, axis=1)
    y_train = train_final[target_col]
    X_test = test_final.copy()
    
    print(f"\n✅ DADOS PRONTOS PARA MODELAGEM:")
    print(f"   X_train: {X_train.shape}")
    print(f"   y_train: {y_train.shape}")
    print(f"   X_test: {X_test.shape}")
    
    # Verificar distribuição do target
    print(f"\n🎯 Distribuição do target:")
    print(y_train.value_counts())
    print(f"   Proporção: {y_train.value_counts(normalize=True)}")
    
    # Verificar se há desbalanceamento
    if len(y_train.value_counts()) == 2:
        minority_class = y_train.value_counts().min()
        majority_class = y_train.value_counts().max()
        ratio = minority_class / majority_class
        
        if ratio < 0.3:
            print(f"⚠️  ATENÇÃO: Classes desbalanceadas (ratio: {ratio:.3f})")
            print("   Considere usar técnicas de balanceamento ou métricas adequadas")
        else:
            print(f"✅ Classes razoavelmente balanceadas (ratio: {ratio:.3f})")

else:
    print("❌ PROBLEMA: Target não identificado - verifique os dados")

=== VERIFICAÇÃO FINAL DOS DADOS PROCESSADOS ===
📏 Train shape: (646, 33)
📏 Test shape: (277, 32)
🎯 Target identificado: labels
🔄 Colunas em comum: 32
❌ Só no train: set()
❌ Só no test: set()

🔍 Valores nulos train: 0
🔍 Valores nulos test: 0

📊 Tipos de dados train:
float64    31
int32       1
int64       1
Name: count, dtype: int64

✅ DADOS PRONTOS PARA MODELAGEM:
   X_train: (646, 32)
   y_train: (646,)
   X_test: (277, 32)

🎯 Distribuição do target:
labels
1    418
0    228
Name: count, dtype: int64
   Proporção: labels
1    0.647059
0    0.352941
Name: proportion, dtype: float64
✅ Classes razoavelmente balanceadas (ratio: 0.545)


In [85]:
# MODELAGEM - MÚLTIPLOS ALGORITMOS
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# 1. Dividir dados para validação
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"📊 Divisão dos dados:")
print(f"   Treino: {X_train_split.shape}")
print(f"   Validação: {X_val.shape}")

# 2. Testar múltiplos modelos
modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

resultados = {}

for nome, modelo in modelos.items():
    print(f"\n🔧 Treinando {nome}...")
    
    # Cross-validation
    cv_scores = cross_val_score(modelo, X_train_split, y_train_split, 
                               cv=5, scoring='roc_auc')
    
    # Treinar modelo completo
    modelo.fit(X_train_split, y_train_split)
    
    # Predições
    y_pred = modelo.predict(X_val)
    y_pred_proba = modelo.predict_proba(X_val)[:, 1]
    
    # Métricas
    auc = roc_auc_score(y_val, y_pred_proba)
    
    resultados[nome] = {
        'modelo': modelo,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'auc_val': auc
    }
    
    print(f"   CV AUC: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"   Val AUC: {auc:.3f}")

# 3. Escolher melhor modelo
melhor_modelo = max(resultados.keys(), key=lambda x: resultados[x]['auc_val'])
print(f"\n🏆 Melhor modelo: {melhor_modelo}")
print(f"   AUC: {resultados[melhor_modelo]['auc_val']:.3f}")

📊 Divisão dos dados:
   Treino: (516, 32)
   Validação: (130, 32)

🔧 Treinando Random Forest...
   CV AUC: 0.807 ± 0.056
   Val AUC: 0.803

🔧 Treinando Gradient Boosting...
   CV AUC: 0.776 ± 0.060
   Val AUC: 0.824

🔧 Treinando Logistic Regression...
   CV AUC: 0.763 ± 0.061
   Val AUC: 0.770

🏆 Melhor modelo: Gradient Boosting
   AUC: 0.824


In [86]:
# MODELAGEM - MÚLTIPLOS ALGORITMOS COM MÉTRICAS COMPLETAS
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# 1. Dividir dados para validação
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"📊 Divisão dos dados:")
print(f"   Treino: {X_train_split.shape}")
print(f"   Validação: {X_val.shape}")

# 2. Testar múltiplos modelos
modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

resultados = {}
metricas_comparacao = []

for nome, modelo in modelos.items():
    print(f"\n🔧 Treinando {nome}...")
    
    # Cross-validation
    cv_scores = cross_val_score(modelo, X_train_split, y_train_split, 
                               cv=5, scoring='roc_auc')
    
    # Treinar modelo completo
    modelo.fit(X_train_split, y_train_split)
    
    # Predições
    y_pred = modelo.predict(X_val)
    y_pred_proba = modelo.predict_proba(X_val)[:, 1]
    
    # Calcular todas as métricas
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Armazenar resultados
    resultados[nome] = {
        'modelo': modelo,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'auc_val': auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    # Para tabela comparativa
    metricas_comparacao.append({
        'Modelo': nome,
        'CV AUC': f"{cv_scores.mean():.3f} ± {cv_scores.std():.3f}",
        'Val AUC': f"{auc:.3f}",
        'Acurácia': f"{accuracy:.3f}",
        'Precisão': f"{precision:.3f}",
        'Recall': f"{recall:.3f}",
        'F1-Score': f"{f1:.3f}"
    })
    
    print(f"   CV AUC: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"   Val AUC: {auc:.3f}")
    print(f"   Acurácia: {accuracy:.3f}")
    print(f"   Precisão: {precision:.3f}")
    print(f"   Recall: {recall:.3f}")
    print(f"   F1-Score: {f1:.3f}")

# 3. Tabela comparativa das métricas
print("\n" + "="*80)
print("📊 TABELA COMPARATIVA DE MÉTRICAS")
print("="*80)

df_metricas = pd.DataFrame(metricas_comparacao)
print(df_metricas.to_string(index=False))

# 4. Escolher melhor modelo (baseado em AUC)
melhor_modelo_nome = max(resultados.keys(), key=lambda x: resultados[x]['auc_val'])
melhor_resultado = resultados[melhor_modelo_nome]

print(f"\n🏆 MELHOR MODELO: {melhor_modelo_nome}")
print(f"   AUC: {melhor_resultado['auc_val']:.3f}")
print(f"   Acurácia: {melhor_resultado['accuracy']:.3f}")
print(f"   F1-Score: {melhor_resultado['f1_score']:.3f}")

# 5. Relatório detalhado do melhor modelo
print(f"\n📋 RELATÓRIO DETALHADO - {melhor_modelo_nome}")
print("="*50)

melhor_modelo_obj = melhor_resultado['modelo']
y_pred_melhor = melhor_modelo_obj.predict(X_val)

# Classification Report
print("\n📈 Classification Report:")
print(classification_report(y_val, y_pred_melhor, target_names=['Fracasso', 'Sucesso']))

# Matriz de Confusão
print("\n🔍 Matriz de Confusão:")
cm = confusion_matrix(y_val, y_pred_melhor)
print(f"                  Predito")
print(f"              Fracasso  Sucesso")
print(f"Real Fracasso     {cm[0,0]:3d}      {cm[0,1]:3d}")
print(f"     Sucesso      {cm[1,0]:3d}      {cm[1,1]:3d}")

# Interpretação da matriz
vn, fp, fn, vp = cm.ravel()
print(f"\n📊 Interpretação:")
print(f"   ✅ Verdadeiros Negativos (Fracasso previsto corretamente): {vn}")
print(f"   ❌ Falsos Positivos (Fracasso previsto como Sucesso): {fp}")
print(f"   ❌ Falsos Negativos (Sucesso previsto como Fracasso): {fn}")
print(f"   ✅ Verdadeiros Positivos (Sucesso previsto corretamente): {vp}")

# 6. Análise de importância das features (se for Random Forest ou Gradient Boosting)
if melhor_modelo_nome in ['Random Forest', 'Gradient Boosting']:
    print(f"\n🎯 TOP 10 FEATURES MAIS IMPORTANTES ({melhor_modelo_nome}):")
    print("="*60)
    
    importances = melhor_modelo_obj.feature_importances_
    feature_names = X_train.columns
    
    # Criar DataFrame com importâncias
    df_importances = pd.DataFrame({
        'Feature': feature_names,
        'Importancia': importances
    }).sort_values('Importancia', ascending=False)
    
    # Mostrar top 10
    top_features = df_importances.head(10)
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        print(f"{i:2d}. {row['Feature']:25s}: {row['Importancia']:.4f}")


📊 Divisão dos dados:
   Treino: (516, 32)
   Validação: (130, 32)

🔧 Treinando Random Forest...
   CV AUC: 0.807 ± 0.056
   Val AUC: 0.803
   Acurácia: 0.777
   Precisão: 0.784
   Recall: 0.905
   F1-Score: 0.840

🔧 Treinando Gradient Boosting...
   CV AUC: 0.776 ± 0.060
   Val AUC: 0.824
   Acurácia: 0.785
   Precisão: 0.798
   Recall: 0.893
   F1-Score: 0.843

🔧 Treinando Logistic Regression...
   CV AUC: 0.763 ± 0.061
   Val AUC: 0.770
   Acurácia: 0.708
   Precisão: 0.761
   Recall: 0.798
   F1-Score: 0.779

📊 TABELA COMPARATIVA DE MÉTRICAS
             Modelo        CV AUC Val AUC Acurácia Precisão Recall F1-Score
      Random Forest 0.807 ± 0.056   0.803    0.777    0.784  0.905    0.840
  Gradient Boosting 0.776 ± 0.060   0.824    0.785    0.798  0.893    0.843
Logistic Regression 0.763 ± 0.061   0.770    0.708    0.761  0.798    0.779

🏆 MELHOR MODELO: Gradient Boosting
   AUC: 0.824
   Acurácia: 0.785
   F1-Score: 0.843

📋 RELATÓRIO DETALHADO - Gradient Boosting

📈 Classificat

In [ ]:
# SUBMISSÃO FINAL - GRADIENT BOOSTING ORIGINAL
print("📝 GERANDO SUBMISSÃO COM GRADIENT BOOSTING")
print("="*55)

# Usar o Gradient Boosting original (melhor modelo básico)
modelo_final = resultados['Gradient Boosting']['modelo']
auc_final = resultados['Gradient Boosting']['auc_val']

print(f"🏆 Modelo escolhido: Gradient Boosting Original")
print(f"📊 AUC de validação: {auc_final:.4f}")

# Treinar modelo final em TODOS os dados de treino
print(f"\nTreinando modelo final em todos os dados...")
print(f"   X_train: {X_train.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   X_test: {X_test.shape}")

modelo_final.fit(X_train, y_train)
print("✅ Modelo treinado com sucesso!")

# Fazer predições no test set
print("Gerando predições no test set...")
# Predições binárias (0 ou 1) - não probabilidades
predicoes_finais = modelo_final.predict(X_test)

# Também calcular probabilidades para análise
probabilidades = modelo_final.predict_proba(X_test)[:, 1]

print(f"✅ Predições geradas: {len(predicoes_finais)} valores binários (0 ou 1)")

# Verificar coluna ID
if 'id' in test.columns:
    id_col = test['id']
    print("✅ Usando coluna 'id' do dataset")
else:
    id_col = range(len(test))
    print("⚠️ Criando IDs sequenciais")

# Criar arquivo de submissão com valores binários
submission = pd.DataFrame({
    'id': id_col,
    'labels': predicoes_finais  # 0 ou 1
})

# Salvar submission
nome_arquivo = 'submission_gradient_boosting.csv'
submission.to_csv(nome_arquivo, index=False)
print(f"✅ Arquivo '{nome_arquivo}' salvo!")

# Mostrar informações da submissão
print(f"\n📋 INFORMAÇÕES DA SUBMISSÃO:")
print(f"   Linhas: {len(submission)}")
print(f"   Colunas: {list(submission.columns)}")
print(f"   Valores únicos: {sorted(submission['labels'].unique())}")
print("\nPrimeiras 10 linhas:")
print(submission.head(10))

# Estatísticas das predições BINÁRIAS
print(f"\n📈 ESTATÍSTICAS DAS PREDIÇÕES (0/1):")
successo_predito = (predicoes_finais == 1).sum()
fracasso_predito = (predicoes_finais == 0).sum()
total = len(predicoes_finais)

print(f"   Total de predições: {total}")
print(f"   Fracasso (0): {fracasso_predito} ({fracasso_predito/total*100:.1f}%)")
print(f"   Sucesso (1): {successo_predito} ({successo_predito/total*100:.1f}%)")

# Estatísticas das probabilidades (para análise)
print(f"\n📊 ESTATÍSTICAS DAS PROBABILIDADES (para análise):")
print(f"   Média das probabilidades: {probabilidades.mean():.4f}")
print(f"   Mediana das probabilidades: {np.median(probabilidades):.4f}")
print(f"   Min: {probabilidades.min():.4f}")
print(f"   Max: {probabilidades.max():.4f}")

# Comparar com distribuição do treino
train_success_rate = y_train.mean()
test_success_rate = successo_predito / total

print(f"\n🔍 COMPARAÇÃO COM TREINO:")
print(f"   Taxa de sucesso no treino: {train_success_rate:.1%}")
print(f"   Taxa de sucesso predita no teste: {test_success_rate:.1%}")
print(f"   Diferença: {test_success_rate - train_success_rate:+.1%}")

if abs(test_success_rate - train_success_rate) < 0.1:
    print("✅ Distribuições similares - predições consistentes!")
elif test_success_rate > train_success_rate + 0.1:
    print("📈 Modelo prevê mais sucessos que o observado no treino")
else:
    print("📉 Modelo prevê menos sucessos que o observado no treino")

# Verificar threshold usado pelo modelo
threshold_usado = 0.5
acima_threshold = (probabilidades > threshold_usado).sum()
print(f"\n🎯 ANÁLISE DO THRESHOLD:")
print(f"   Threshold padrão usado: {threshold_usado}")
print(f"   Probabilidades > {threshold_usado}: {acima_threshold}")
print(f"   Predições como sucesso (1): {successo_predito}")
print(f"   ✅ Consistência: {'OK' if acima_threshold == successo_predito else 'ERRO'}")

print(f"\n🎉 SUBMISSÃO PRONTA!")
print(f"📁 Arquivo: {nome_arquivo}")
print(f"🏆 Modelo: Gradient Boosting Original")
print(f"📊 AUC esperado: {auc_final:.4f}")
print(f"🎯 Formato: Valores binários (0=Fracasso, 1=Sucesso)")
print(f"💡 Pronto para upload no Kaggle!")

📝 GERANDO SUBMISSÃO COM GRADIENT BOOSTING
🏆 Modelo escolhido: Gradient Boosting Original
📊 AUC de validação: 0.8230

Treinando modelo final em todos os dados...
   X_train: (646, 32)
   y_train: (646,)
   X_test: (277, 32)
✅ Modelo treinado com sucesso!
Gerando predições no test set...
✅ Predições geradas: 277 valores binários (0 ou 1)
✅ Usando coluna 'id' do dataset
✅ Arquivo 'submission_gradient_boosting.csv' salvo!

📋 INFORMAÇÕES DA SUBMISSÃO:
   Linhas: 277
   Colunas: ['id', 'labels']
   Valores únicos: [0, 1]

Primeiras 10 linhas:
    id  labels
0   70       1
1   23       0
2  389       1
3  872       1
4  920       0
5  690       1
6  588       0
7  144       0
8  875       1
9  900       1

📈 ESTATÍSTICAS DAS PREDIÇÕES (0/1):
   Total de predições: 277
   Fracasso (0): 88 (31.8%)
   Sucesso (1): 189 (68.2%)

📊 ESTATÍSTICAS DAS PROBABILIDADES (para análise):
   Média das probabilidades: 0.6149
   Mediana das probabilidades: 0.7283
   Min: 0.0085
   Max: 0.9856

📊 DISTRIBUIÇÃO D